In [1]:
pip install -e .

Obtaining file:///Users/jkent/Documents/intake-stac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Attempting uninstall: intake-stac
    Found existing installation: intake-stac 0.3.0.post80+dirty
    Uninstalling intake-stac-0.3.0.post80+dirty:
      Successfully uninstalled intake-stac-0.3.0.post80+dirty
  Running setup.py develop for intake-stac
Note: you may need to restart the kernel to use updated packages.


In [2]:
import intake
from deepdiff import DeepDiff
import yaml
import pystac
import warnings
warnings.filterwarnings('ignore')
import intake_stac
intake_stac.__version__

'0.3.0.post80+dirty'

In [3]:
item_url = str('intake_stac/tests/data/1.0.0/collection/simple-item.json')
pystac_item = pystac.Item.from_file(item_url)

cat1 = intake_stac.StacItem(pystac_item)
cat1_str = cat1.yaml() #This is the YAML method we implemented

d = yaml.load(cat1_str)

In [4]:
# Test Cat Item YAML
for key in ['bbox','date','datetime','geometry']: #,'version']:
    assert key in d['metadata']
for key in ['B02','B03']:
    assert key in d['sources']

In [5]:
#Test Cat Item YAML Roundtrip
test_file = 'test.yaml'
with open(test_file, 'w') as f:
    f.write(cat1_str)

cat2 = intake.open_catalog(test_file)

for key in ['B02','B03']:
    assert key in cat2
        
assert cat1.metadata == cat2.metadata
assert set(cat1.walk()) == set(cat2.walk())

keys = ['B02', 'B03']
for k in keys:
    assert k in cat1
    assert k in cat2
    assert type(cat1[k]) == type(cat2[k])

    #cat2[k].metadata.pop('catalog_dir', None)
    assert set(cat1[k].metadata) == set(cat2[k].metadata), print(cat1[k].metadata, cat2[k].metadata)
    for j in set(cat2[k].metadata):
        if j == 'catalog_dir':
            continue
        assert cat1[k].metadata[j] == cat1[k].metadata[j]

    assert set(cat1[k].describe()) == set(cat2[k].describe())
    for j in set(cat1[k].describe()):
        assert cat1[k].describe()[j] == cat2[k].describe()[j]

    assert set(cat1.walk()[k].describe()) == set(cat2.walk()[k].describe())
    for j in set(cat1.walk()[k].describe()):
        assert cat1.walk()[k].describe()[j] == cat2.walk()[k].describe()[j]

{'href': 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/17/Q/LA/2017/12/27/S2B_MSIL2A_20171227T160459_N0212_R054_T17QLA_20201014T165101.SAFE/GRANULE/L2A_T17QLA_A004227_20171227T160750/IMG_DATA/R10m/T17QLA_20171227T160459_B02_10m.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'Band 2 - Blue', 'eo:bands': [{'name': 'B02', 'common_name': 'blue', 'description': 'Band 2 - Blue', 'center_wavelength': 0.49, 'full_width_half_max': 0.098}], 'gsd': 10, 'proj:shape': [10980, 10980], 'proj:bbox': [300000, 1990200, 409800, 2100000], 'proj:transform': [10, 0, 300000, 0, -10, 2100000], 'roles': ['data'], 'plots': {'geotiff': {'kind': 'image', 'x': 'x', 'y': 'y', 'frame_width': 500, 'data_aspect': 1, 'rasterize': True, 'dynamic': True, 'cmap': 'viridis'}}} {'eo:bands': [{'center_wavelength': 0.49, 'common_name': 'blue', 'description': 'Band 2 - Blue', 'full_width_half_max': 0.098, 'name': 'B02'}], 'gsd': 10, 'href': 'https://sentinel2l2a01.blob.core.wind

AssertionError: None

In [9]:
key_diff = set(cat2.walk()) - set(cat1.walk())
print(key_diff)

value_diff = { k : cat2.walk()[k] for k in set(cat2.walk()) - set(cat1.walk()) }
print(value_diff)

cat2.walk() == cat1.walk()

#diff = DeepDiff(cat1.walk(), cat2.walk())
#diff

#type(cat1['B02'])
#type(cat2['B02'])
#type(cat1.walk()['B02'])
#type(cat2.walk()['B02'])

set()
{}


False

In [6]:
#cat1['B02'].metadata == cat2['B02'].metadata # False
assert set(cat1['B03'].metadata) == set(cat2['B03'].metadata)

print(set(cat1['B03'].metadata)) 
print(set(cat2['B03'].metadata)) 

#cat1['B02'].metadata['catalog_dir'] == cat2['B02'].metadata['catalog_dir'] # False
#print(cat1['B02'].metadata['catalog_dir']) # ''
#print(cat2['B02'].metadata['catalog_dir']) # /Users/jkent/Documents/intake-stac/

#for j in set(cat1['B02'].metadata):
#    print(j)

{'roles', 'proj:bbox', 'title', 'catalog_dir', 'plots', 'href', 'proj:transform', 'type', 'gsd', 'eo:bands', 'proj:shape'}
{'roles', 'proj:bbox', 'title', 'catalog_dir', 'plots', 'href', 'proj:transform', 'type', 'gsd', 'eo:bands', 'proj:shape'}
